# Desenvolvimento Embarcado em Inteligência de Negócio 🧠

## Sumário da Aula

<ul>
    <li>Configuração ⚙️</li>
    <ul>
        <li>Power BI 📊</li>
        <li>Power BI 📊 + Python 🐍</li>
    </ul>
    <li>Aspectos de Desenvolvimento Embarcado 🧠</li>
    <ul>
        <li>Montando a Raspagem de Dados em Python 🐍 para o Power BI 📊</li>
        <li>Executando a Raspagem de Dados com Python 🐍 no Power BI 📊</li>
    </ul>
</ul>

### Biblioteca Principal 📚: Não se aplica

## Configuração ⚙️

### Power BI 📊

"Microsoft Power BI is an interactive data visualization software product developed by Microsoft with a primary focus on business intelligence."

<img src="https://upload.wikimedia.org/wikipedia/commons/c/cf/New_Power_BI_Logo.svg" width="100" style="float: right;">

👉 Faça a Instalação: <a href='https://powerbi.microsoft.com/pt-br/downloads/'>powerbi.microsoft.com</a>

👉 Power BI <a href='https://s3.amazonaws.com/assets.datacamp.com/email/other/Power+BI_Cheat+Sheet.pdf'>cheat sheet</a>

### Power BI 📊 + Python 🐍

<pre>É possível executar scripts do Python diretamente no Power BI</pre>

<pre>A integração do Power BI com o Python exige a instalação do Python e dos pacotes <i>pandas</i> e <i>matplotlib</i> do Python.</pre>

<pre>Aproveitaremos para instalar as bibliotecas que serão necessárias no Estudo de Caso desta aula: <i>BeautifulSoupt</i>, <i>requests</i> e <i>urllib3</i></pre>

<img src='./figs/Command.png' width='100%'>

<pre>No Power BI, para habilitar o script Python no Power BI, selecione Arquivo > Opções e Configurações > Opções</pre>

<img src='figs/PowerBI01.png' width='100%'>

<img src='figs/PowerBI01Config01.png' width='100%'>

<pre>Em Opções > Scripts do Python, forneça ou edite o caminho de instalação local do Python. Selecione Ok.</pre>

<img src='figs/PowerBI01Config02.png' width='100%'>

## Aspectos de Desenvolvimento Embarcado 🧠

👉 No Power BI, para executar um script em Python com os dados correntes, o Power BI armazena os dados correntes no DataFrame <code>dataset</code>. Veja:

<img src='figs/PowerBI00.png'>

<pre>Portanto, se quisermos trabalhar com Python nos dados correntes do Power BI, partimos do uso de um DataFrame 'dataset'</pre>

### Montando a Raspagem de Dados em Python 🐍 para o Power BI 📊

<pre>O código abaixo é conhecido nosso, da aula de Limpeza e Preparação dos Dados 🏗️ 🎲</pre>

<pre>Ele recupera os dados da tabela de códigos das UF (obtidos de uma API do IBGE) e que estão salvos no nosso sistema de arquivos</pre>

<pre>Nós armazenaremos os dados em um DataFrame 'dataset' para simular que estamos usando os dados de uma tabela corrente no Power BI</pre>

In [1]:
import os, json, pandas as pd

def build_path(subfolder = 'raw'):
    folderpath = os.path.join(os.getcwd(), os.pardir, 
                              'project', 'data', subfolder)
    folderpath = os.path.abspath(folderpath)
    if not os.path.exists(folderpath): 
        os.makedirs(folderpath)  
    return folderpath

def get_estados_codigos(filename='estados_codigos.json'):
    filepath = os.path.join(build_path(), filename)
    with open(filepath) as jsonfile:
        return pd.json_normalize(json.load(jsonfile))
    
dataset = get_estados_codigos()

dataset.head(3)

,id,sigla,nome,regiao.id,regiao.sigla,regiao.nome
0,11,RO,Rondônia,1,N,Norte
1,12,AC,Acre,1,N,Norte
2,13,AM,Amazonas,1,N,Norte


<pre>O código abaixo é outro conhecido nosso, da Raspagem Avançada de Dados ✂️ (Fontes de Dados 🚰 🎲)</pre>

<pre>Ele percorre uma lista de siglas do DataFrame de códigos da UF e raspa os dados de um site do IBGE a cada sigla lida</pre>

<pre>Nós utilizaremos este código para transformar os dados do DataFrame armazenado na variável 'dataset' dentro do Power BI</pre>

In [2]:
from bs4 import BeautifulSoup
import requests, urllib3, ssl
import pandas as pd

class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)
    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)
        
def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

def parse_indicators(indicators):
    label = indicators.find('div', class_='ind-label')
    value = indicators.find('p', class_='ind-value').get_text(strip=False)
    measure = indicators.find('span', class_='indicador-unidade')
    measure = measure.get_text(strip=True) if measure else '\xa0'

    return {'label': label.get_text(strip=True), \
            'value': [x.strip() for x in value.split(measure)][0], \
            'measure': measure.strip()}

def scrape_from_internet(BASE_URI, lower_state):
    BASE_PATH_STATE = f'cidades-e-estados/{lower_state}.html'
    URL_STATE = f'{BASE_URI}/{BASE_PATH_STATE}'
    try:
        response = requests.get(f'{URL_STATE}')
    except requests.exceptions.SSLError:
        response = get_legacy_session().get(f'{URL_STATE}')
    return response

def parse(html):
    list_of_indicators = []
    if html:
        indicators_soup = BeautifulSoup(html, 'html.parser')
        all_indicators = indicators_soup.find_all('div', class_='indicador')
        for indicators in all_indicators:
            list_of_indicators.append(parse_indicators(indicators))
    return list_of_indicators

def states_indicators(BASE_URI, json_states):
    json_states_characteristics = []
    for json_state in json_states:
        lower_state = json_state['sigla'].lower()
        json_states_characteristics.append(\
            {'state': lower_state,\
             'characteristics': parse(scrape_from_internet(BASE_URI, lower_state).content)})
    return json_states_characteristics

BASE_URI = 'https://ibge.gov.br'

<pre>Inicialmente, vamos utilizar a tabela corrente do Power BI (<i>dataset</i>) e

transformar no formato da entrada aceita pelo código da Raspagem Avançada de Dados</pre>

In [3]:
siglas = [{'sigla': state} for state in dataset['sigla']]
siglas[:3]

[{'sigla': 'RO'}, {'sigla': 'AC'}, {'sigla': 'AM'}]

<pre>Agora, vamos reutilizar nosso código da Raspagem Avançada de Dados para obter os dados que desejamos</pre>

In [ ]:
dados = states_indicators(BASE_URI, siglas)
dados[0]

<pre>Neste momento, vamos normalizar nosso json para obter um DataFrame com todos os dados que precisamos</pre>

In [ ]:
dados = pd.json_normalize(dados, record_path='characteristics', 
                          record_prefix='characteristics_', 
                          meta='state', meta_prefix='state_')
dados = dados.drop(columns='characteristics_measure')
dados

<pre>Por fim, vamos transformar nossos dados com Pivoteamento do Formato Longo em Formato Largo</pre>

In [ ]:
dados = dados.pivot(columns='characteristics_label', 
                    index='state_state')
dados.head(3)

<pre>Sendo este o resultado que desejamos obter no Power BI, vamos condensar o código que realiza a raspagem de dados e a transformação, 

pois ele será usado na funcionalidade correspondente de transformação de dados do Power BI com script Python

In [ ]:
from bs4 import BeautifulSoup
import requests, urllib3, ssl
import pandas as pd

class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)
    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)
        
def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

def parse_indicators(indicators):
    label = indicators.find('div', class_='ind-label')
    value = indicators.find('p', class_='ind-value').get_text(strip=False)
    measure = indicators.find('span', class_='indicador-unidade')
    measure = measure.get_text(strip=True) if measure else '\xa0'

    return {'label': label.get_text(strip=True), \
            'value': [x.strip() for x in value.split(measure)][0], \
            'measure': measure.strip()}

def scrape_from_internet(BASE_URI, lower_state):
    BASE_PATH_STATE = f'cidades-e-estados/{lower_state}.html'
    URL_STATE = f'{BASE_URI}/{BASE_PATH_STATE}'
    try:
        response = requests.get(f'{URL_STATE}')
    except requests.exceptions.SSLError:
        response = get_legacy_session().get(f'{URL_STATE}')
    return response

def parse(html):
    list_of_indicators = []
    if html:
        indicators_soup = BeautifulSoup(html, 'html.parser')
        all_indicators = indicators_soup.find_all('div', class_='indicador')
        for indicators in all_indicators:
            list_of_indicators.append(parse_indicators(indicators))
    return list_of_indicators

def states_indicators(BASE_URI, json_states):
    json_states_characteristics = []
    for json_state in json_states:
        lower_state = json_state['sigla'].lower()
        json_states_characteristics.append(\
            {'state': lower_state,\
             'characteristics': parse(scrape_from_internet(BASE_URI, lower_state).content)})
    return json_states_characteristics

BASE_URI = 'https://ibge.gov.br'

siglas = [{'sigla': state} for state in dataset['sigla']]
dados = states_indicators(BASE_URI, siglas)
dados = pd.json_normalize(dados, record_path='characteristics', record_prefix='characteristics_', meta='state', meta_prefix='state_')
dados = dados.drop(columns='characteristics_measure')
dados = dados.pivot(columns='characteristics_label', index='state_state')

### Executando a Raspagem de Dados com Python 🐍 no Power BI 📊

<pre>Para executar o script Python, no menu Página Inicial da faixa de opções do Power BI, selecione Obter dados.</pre>

<pre>Vamos escolher a opção <code>Web</code> e obter os códigos de UF a partir da API do IBGE, que utilizaremos na nossa raspagem.</pre>

<img src='figs/PowerBI03.png' width='100%'>

<pre>Insira o endereço da API do IBGE e selecione Ok. Caso pergunte sobre autenticação, selecione o modo Anônimo.</pre>

<img src='figs/PowerBI04.png' width='100%'>

<pre>Veja que a consulta à API retornou o mesmo DataFrame de códigos das UF que usamos mais cedo nesta aula</pre>

<pre>Para embarcar o script em Python no PowerBI e transformar este DataFrame, vá em Transformar > Executar script Python</pre>

<img src='figs/PowerBI05.png' width='100%'>

<pre>Veja que um modal foi aberto para inserir o script em Python.</pre>

<pre>O modal informa a variável que armazena o DataFrame: ('dataset'), bem como o caminho do executável do Python que será utilizado</pre>

<img src='figs/PowerBI06.png' width='100%'>

<pre>Insira o script integralmente, tal como está na última célula imediatamente anterior a esta seção</pre>

<pre>Observe que o DataFrame referenciado pela variável 'dados' armazenará o resultado da nossa transformação com Python</pre>

<img src='figs/PowerBI07.png' width='100%'>

<pre>Imediatamente após a execução do script, aparecem dois DataFrames: 
<ul>
    <li>'dataset': a entrada do nosso script em Python.
    <li>'dados': a saída do nosso script em Python; e
</ul></pre>

<pre>Selecione a 'Table' correspondente à variável 'dados', para resgatar o DataFrame resultante da nossa transformação</pre>

<img src='figs/PowerBI08.png' width='100%'>

<pre>Remova a etapa de alteração de tipos de dados inserida automaticamente pelo Power BI e use a primeira linha do DataFrame como cabeçalho</pre>

<pre>Automaricamente, o Power BI detectará os tipos de dados corretos das nossas colunas, de forma semelhante ao Pandas</pre>

<img src='figs/PowerBI09.png' width='100%'>

<pre>Veja que o DataFrame resultante está com os dados corretos, com o cabeçalho correto e com os tipos de dados corretos.</pre>

<pre>Feche e aplique as alterações.</pre>

<img src='figs/PowerBI10.png' width='100%'>

<pre>Na sequência, o Power BI importará os dados transformados pelo script para o aplicativo principal.</pre>

👉 Para atualizar os dados, selecione Atualizar no menu Página Inicial da faixa de opções do Power BI. Quando você atualiza um script, o Power BI executa o script do Python novamente.

<img src='figs/PowerBI11.png' width='100%'>

<pre>Se o script for executado com êxito, o Power BI importa os dados transformados pelo script.</pre>

<img src='figs/PowerBI12.png' width='100%'>

<pre>Você pode utilizar os dados transformados para criar visualizações e relatórios.</pre>

<pre>Rapidamente, podemos criar um gráfico de dispersão que contrasta o IDH e o Rendimento mensal domiciliar per capita das UF do Brasil</pre>

<img src='figs/PowerBI13.png' width='100%'>

<img src='figs/PowerBI14.png' width='100%'>

## Saiba Mais 👩‍🏫

👉 <a href='https://suap.enap.gov.br/vitrine/curso/2016/?area=2'>Praticando Business Intelligence</a>